# Multimethod polymorphism

## Polymorphism and its types

### Ad-hoc polymorphism

"closed-dispatch" polymorphism

In [1]:
; A function that ckecks the type of 'thing' argument 
; to call different functions based on the type

(defn ad-hoc-type-namer [thing]
    (condp = (type thing) 
        java.lang.String "string" 
        clojure.lang.PersistentVector "vector"))


#'user/ad-hoc-type-namer

In [2]:
; Passing a string parameter

(ad-hoc-type-namer "I'm a string")

"string"

In [3]:
; Passing a vector parameter

(ad-hoc-type-namer [])

"vector"

In [4]:
; If there’s a type this function doesn’t know how to handle, an exception is thrown

(ad-hoc-type-namer {}) 

Execution error (IllegalArgumentException) at user/ad-hoc-type-namer (REPL:5).
No matching clause: class clojure.lang.PersistentArrayMap


class java.lang.IllegalArgumentException: 

"open-dispatch" polymorphism

In [5]:
; Pull type implementations out into a separate map

(def type-namer-implementations 
    {java.lang.String (fn [thing] "string")
     clojure.lang.PersistentVector (fn [thing] "vector")})


#'user/type-namer-implementations

In [6]:
; A function that ckecks the type of 'thing' argument 
; to call different functions based on the type, 
; without a fixed set of types to ckeck
 
(defn open-ad-hoc-type-namer [thing]
    (let [dispatch-value (type thing)] 
        ; Use a dispatch value as key to implementation map
        (if-let [implementation (get type-namer-implementations dispatch-value)] 
            (implementation thing) 
            (throw (IllegalArgumentException. (str "No implementation found for " dispatch-value))))))


#'user/open-ad-hoc-type-namer

In [7]:
; Passing a parameter of string type

(open-ad-hoc-type-namer "I'm a string") 

"string"

In [8]:
; Passing a parameter of vector type

(open-ad-hoc-type-namer []) 

"vector"

In [9]:
; Passing a parameter of map type (it will throw an exception)

(open-ad-hoc-type-namer {}) 

Execution error (IllegalArgumentException) at user/open-ad-hoc-type-namer (REPL:10).
No implementation found for class clojure.lang.PersistentArrayMap


class java.lang.IllegalArgumentException: 

In [10]:
; Adding a 'map' type to a the map of types

(def type-namer-implementations 
    (assoc type-namer-implementations clojure.lang.PersistentArrayMap (fn [thing] "map")))


#'user/type-namer-implementations

In [11]:
; Passing a parameter of map type again (it won't throw an exception now)

(open-ad-hoc-type-namer {})

"map"

### Subtype polymorphism

Checking types with ad-hoc polymorphism

In [12]:
; A function that checks if its argument is map-like

(defn map-type-namer [thing] 
    (condp = (type thing)
        clojure.lang.PersistentArrayMap "map"
        clojure.lang.PersistentHashMap "map")) ; Notice the code duplication

#'user/map-type-namer

In [13]:
; Passing a parameter of hash-map type (recognized)

(map-type-namer (hash-map))

"map"

In [14]:
; Passing a parameter of array-map type (recognized)

(map-type-namer (array-map))

"map"

In [15]:
; Passing a parameter of sorted-map type (unrecognized)

(map-type-namer (sorted-map)) 

Execution error (IllegalArgumentException) at user/map-type-namer (REPL:4).
No matching clause: class clojure.lang.PersistentTreeMap


class java.lang.IllegalArgumentException: 

Checking types with subtype polymorphism

In [16]:
(defn subtyping-map-type-namer [thing] 
    (cond
        (instance? clojure.lang.APersistentMap thing) "map" ; APersistent-Map is Java superclass of all map-like things in Clojure
         :else (throw (IllegalArgumentException. (str "No implementation found for ") (type thing)))))


#'user/subtyping-map-type-namer

In [17]:
; Passing a parameter of hash-map type (recognized)

(subtyping-map-type-namer (hash-map))

"map"

In [18]:
; Passing a parameter of array-map type (recognized)

(subtyping-map-type-namer (array-map))

"map"

In [19]:
; Passing a parameter of sorted-map type (recognized now)

(subtyping-map-type-namer (sorted-map)) 

"map"

## Polymorphism using multimethods

Implementing a tracking expense service without multimethods 

In [20]:
; A mapping for a sample user

(def example-user {:login "rob" 
                   :referrer "mint.com" 
                   :salary 100000})


#'user/example-user

In [6]:
; A function that calculates the fee depending of the referrer type

(defn fee-amount [percentage user]
    (with-precision 16 :rounding HALF_EVEN 
        ; Using BigDecimal when dealing with money
        (* 0.01M percentage (:salary user))))

#'user/fee-amount

In [22]:
; A function that performs close dispatch based on the referrer type

(defn affiliate-fee [user]
    (case (:referrer user)
        "google.com" (fee-amount 0.01M user)
        "mint.com" (fee-amount 0.03M user)
        (fee-amount 0.02M user)))


#'user/affiliate-fee

In [23]:
; Testing the last function with the example-user map

(affiliate-fee example-user)


30.0000M

Implementing a tracking expense service with multimethods 

In [24]:
; A general form of the defmulti macro

"""
(defmulti name docstring? attr-map? dispatch-fn & options)
"""

""

In [25]:
; The defmulti returns a dispatch value 
; with which to find an implementation.

(defmulti affiliate-fee 
    (fn [user] (:referrer user)))


#'user/affiliate-fee

In [26]:
; Defining an implementation for the "mint.com" dispatch value

(defmethod affiliate-fee "mint.com" [user] 
    (fee-amount 0.03M user))

#multifn[affiliate-fee 0x63953eae]

In [27]:
; Defining an implementation for the "google.com" dispatch value

(defmethod affiliate-fee "google.com" [user]
    (fee-amount 0.01M user))

#multifn[affiliate-fee 0x63953eae]

In [28]:
; Defining an implementation for a default dispatch value

(defmethod affiliate-fee :default [user] 
    (fee-amount 0.02M user))

#multifn[affiliate-fee 0x63953eae]

In [29]:
; Testing the first method with the example-user map

(affiliate-fee example-user)

30.0000M

Using multimethods with custom defaults

In [30]:
; Redefining the default case (it won't take effect)

(defmulti affiliate-fee :referrer :default "*")

nil

In [31]:
; Removing the affiliate-fee multimethod from the user namespace

(ns-unmap 'user 'affiliate-fee)

nil

In [32]:
; Redefining the default case (it will take effect now)

(defmulti affiliate-fee :referrer :default "*") 

#'user/affiliate-fee

In [33]:
; Defining the new default case method

(defmethod affiliate-fee "*" [user] 
    (fee-amount 0.02M user))

#multifn[affiliate-fee 0x51b7ab75]

In [34]:
(affiliate-fee example-user)

20.0000M

In [35]:
; A general form of the defmethod macro

"""
(defmethod multifn dispatch-value & fn-tail)
"""


""

In [36]:
; The defmethod macro can also be used with multiple arities

"""
(defmethod my-many-arity-multi :default
    ([] \"no arguments\")
    ([x] \"one argument\")
    ([x & etc] \"many arguments\"))
"""

""

Accessing multimethod map items

In [37]:
; Defining again the method for "mint.com"

(defmethod affiliate-fee "mint.com" [user] 
    (fee-amount 0.03M user))

#multifn[affiliate-fee 0x51b7ab75]

In [38]:
; Defining again the method for "google.com"

(defmethod affiliate-fee "google.com" [user]
    (fee-amount 0.01M user))

#multifn[affiliate-fee 0x51b7ab75]

In [39]:
; Returning all the methods associated to "affiliate-fee"

(methods affiliate-fee) 

{"mint.com" #function[user/eval4178/fn--4179], "*" #function[user/eval4172/fn--4173], "google.com" #function[user/eval4182/fn--4183]}

In [40]:
; Returning the method associated to "mint.com"

(get-method affiliate-fee "mint.com") 

#function[user/eval4178/fn--4179]

In [41]:
; No specific method associated to "example.org"

(get (methods affiliate-fee) "example.org")

nil

In [42]:
; The "*" (default) implementation

(get-method affiliate-fee "example.org") 

#function[user/eval4172/fn--4173]

In [43]:
; The entries in the dispatch map can be called as normal functions.

((get-method affiliate-fee "mint.com") example-user) 

30.0000M

### Multiple dispatch

Creating more mappings of user categories 

In [1]:
(def user-1 {:login "rob" :referrer "mint.com" :salary 100000
             :rating :rating/bronze})
(def user-2 {:login "gordon" :referrer "mint.com" :salary 80000
             :rating :rating/silver})
(def user-3 {:login "kyle" :referrer "google.com" :salary 90000
             :rating :rating/gold})
(def user-4 {:login "celeste" :referrer "yahoo.com" :salary 70000
             :rating :rating/platinum})

#'user/user-4

The new business rules are as follows:

```
Affiliate       Profit rating       Fee (% of salary)

mint.com        Bronze              0.03
mint.com        Silver              0.04
mint.com        Gold/platinum       0.05
google.com      Gold/platinum       0.03
```

In [4]:
; A keyword function that extracts the 2 parameters of interest (referrer and rating)

(defn fee-category [user]
    [(:referrer user) (:rating user)])

(map fee-category [user-1 user-2 user-3 user-4])

(["mint.com" :rating/bronze] ["mint.com" :rating/silver] ["google.com" :rating/gold] ["yahoo.com" :rating/platinum])

In [7]:
; Rewriting the fee-amount function

(defn fee-amount [percentage user]
    (with-precision 16 :rounding HALF_EVEN 
        ; Using BigDecimal when dealing with money
        (* 0.01M percentage (:salary user))))

#'user/fee-amount

In [41]:
; A multimethod that implements the above business rules table

(defmulti profit-based-affiliate-fee 
    fee-category) 

(defmethod profit-based-affiliate-fee ["mint.com" :rating/bronze] [user] 
    (fee-amount 0.03M user))

(defmethod profit-based-affiliate-fee ["mint.com" :rating/silver] [user] 
    (fee-amount 0.04M user))

(defmethod profit-based-affiliate-fee ["mint.com" :rating/gold] [user] 
    (fee-amount 0.05M user))

(defmethod profit-based-affiliate-fee ["mint.com" :rating/platinum] [user] 
    (fee-amount 0.05M user))

(defmethod profit-based-affiliate-fee ["google.com" :rating/gold] [user] 
    (fee-amount 0.03M user))

(defmethod profit-based-affiliate-fee ["google.com" :rating/platinum] [user] 
    (fee-amount 0.03M user))

(defmethod profit-based-affiliate-fee :default [user] 
    (fee-amount 0.02M user))

#multifn[profit-based-affiliate-fee 0x27052c05]

In [31]:
; Testing the multimethod with the 4 initial users

(map profit-based-affiliate-fee [user-1 user-2 user-3 user-4])


(30.0000M 32.0000M 27.0000M 14.0000M)

### Subtype polymorphism using multimethods

Building type hierarchies with 'derive'

In [12]:
; Using :rating/ANY as root type for all ratings

(derive :rating/basic :rating/ANY) 
(derive :rating/premier :rating/ANY)

nil

In [15]:
; Deriving the basic 'class'

(derive :rating/bronze :rating/basic) 
(derive :rating/silver :rating/basic) 


nil

In [16]:
; Deriving the premier 'class'

(derive :rating/gold :rating/premier)
(derive :rating/platinum :rating/premier)


nil

Inspecting the newly created hierarchy with the following keywords

In [17]:
(isa? :rating/gold :rating/premier)

true

In [24]:
; isa? understands transitive relationships, too

(isa? :rating/gold :rating/ANY) 

true

In [19]:
(isa? :rating/ANY :rating/premier)

false

In [25]:
; Types are always kinds of themselves.

(isa? :rating/gold :rating/gold) 

true

In [26]:
; Ancestors returns parents and their parents

(parents :rating/premier) 

#{:rating/ANY}

In [22]:
(ancestors :rating/gold) 

#{:rating/ANY :rating/premier}

In [27]:
; Desendants returns children and their children

(descendants :rating/ANY) 

#{:rating/basic :rating/bronze :rating/gold :rating/premier :rating/silver :rating/platinum}

In [28]:
; Using the last type hierarchy to create a new multimethod

(defmulti greet-user :rating)

(defmethod greet-user :rating/basic [user]
    (str "Hello " (:login user) \.))

(defmethod greet-user :rating/premier [user]
    (str "Welcome, " (:login user) ", valued affiliate member!"))


#multifn[greet-user 0xe4cdab7]

In [32]:
; Testing the new multimethod with the 4 initial users

(map greet-user [user-1 user-2 user-3 user-4])

("Hello rob." "Hello gordon." "Welcome, kyle, valued affiliate member!" "Welcome, celeste, valued affiliate member!")

Refactoring the profit-based-affiliate-fee function to use typw hierarchies and multimethods

In [33]:
; Removing the methods that will be refactored

(remove-method profit-based-affiliate-fee ["mint.com" :rating/gold]) 
(remove-method profit-based-affiliate-fee ["mint.com" :rating/platinum])
(remove-method profit-based-affiliate-fee ["google.com" :rating/gold])
(remove-method profit-based-affiliate-fee ["google.com" :rating/platinum])


#multifn[profit-based-affiliate-fee 0x27052c05]

In [40]:
; Defining the new methods that use the custom type hierarchy

(defmethod profit-based-affiliate-fee ["mint.com" :rating/premier] [user] 
    (fee-amount 0.05M user))

(defmethod profit-based-affiliate-fee ["google.com" :rating/premier] [user] 
    (fee-amount 0.03M user))

#multifn[profit-based-affiliate-fee 0x27052c05]

In [36]:
; Testing the refactored multimethod

(map profit-based-affiliate-fee [user-1 user-2 user-3 user-4]) 

(30.0000M 32.0000M 27.0000M 14.0000M)

Resolving method ambiguities

In [39]:
; Defining a multimethod with potential conflicts

(defmulti size-up (fn [observer observed]
    [(:rating observer) (:rating observed)]))

(defmethod size-up [:rating/platinum :rating/ANY] [_ observed] 
    (str (:login observed) " seems scrawny."))

(defmethod size-up [:rating/ANY :rating/platinum] [_ observed] 
    (str (:login observed) " shimmers with an unearthly light."))

#multifn[size-up 0x7161b55a]

In [42]:
; Testing the above multimethod (it will throw an exception)

(size-up {:rating :rating/platinum} user-4) 

Execution error (IllegalArgumentException) at user/eval4347 (REPL:3).
Multiple methods in multimethod 'size-up' match dispatch value: [:rating/platinum :rating/platinum] -> [:rating/ANY :rating/platinum] and [:rating/platinum :rating/ANY], and neither is preferred


class java.lang.IllegalArgumentException: 

In [44]:
; Resolving the ambiguity with the 'prefer' keyword

(prefer-method size-up [:rating/ANY :rating/platinum] [:rating/platinum :rating/ANY]) 


#multifn[size-up 0x7161b55a]

In [45]:
; Testing the above multimethod (it will work now)

; The method with '[:rating/ANY :rating/platinum]' signature is preferred
(size-up {:rating :rating/platinum} user-4) 

"celeste shimmers with an unearthly light."

In [47]:
; Listing all preferred implementations

(prefers size-up) 

{[:rating/ANY :rating/platinum] #{[:rating/platinum :rating/ANY]}}

User-defined hierarchies

In [57]:
; Hierarchies are just ordinary maps.

(def myhier (make-hierarchy))
myhier 


{:parents {}, :descendants {}, :ancestors {}}

In [60]:
; Adding a new element to a hierarchy don't mutate the original one

(derive myhier :a :letter) 


{:parents {:a #{:letter}}, :ancestors {:a #{:letter}}, :descendants {:letter #{:a}}}

In [61]:
; The original hierarchy is the same

myhier

{:parents {}, :descendants {}, :ancestors {}}

In [62]:
; To create ammutable hierarchy, it should be defined as a Var
 
(def myhier 
    (-> myhier 
        (derive :a :letter) 
        (derive :b :letter)
        (derive :c :letter)))


#'user/myhier

In [63]:
; The isa? method also works with custom hierarchies

(isa? myhier :a :letter) 

true

In [64]:
; The parents method also works with custom hierarchies

(parents myhier :a) 

#{:letter}

In [65]:
; A multimethod that accepts a custom hierarchie as a parameter (as a Var)

(defmulti letter? identity :hierarchy #'myhier) 

(defmethod letter? :letter [_] true)


#multifn[letter? 0x58ec970c]

In [55]:
; Calling the multimethod with an item that doesn't exist in the hierarchy (It will fail)

(letter? :d)

Execution error (IllegalArgumentException) at user/eval4371 (REPL:1).
No method in multimethod 'letter?' for dispatch value: :d


class java.lang.IllegalArgumentException: 

In [66]:
; Adding the missing element and calling the multimethod again (It will work now)

(def myhier (derive myhier :d :letter)) 

(letter? :d)


true